# Time Series Forecasting på ett företags försäljningsprognoser

I det här projektet kommer jag att genomföra en prediktering utav Corporacion Favorita framtida försäljningar. Corporacion Favorita är ett stort detaljhandelsföretag beläget i Ecuador.

### Dataset:

För uppgiften har jag ett dataset som består utav träningsdata, testdata, sample_submission, data om de olika butikerna, samt data om de olika högtiderna i Ecuador.

### Extra information man får från uppgiften är:

*   Löner betalas ut den 15 varje månad vilket kan påverka butikernas försäljning.
*   En jordbävning med magnitud på 7.8 slog till i Ecuador den 16 April 2016, vilket ledde till massor av donationer på vatten, mat och andra "bra-att-ha-artiklar" vilket påverkade detaljhandelsföretag i flera veckor framöver.store-sales-time-series-forecasting

### Material för genomförandet av uppgiften
För att genomföra uppgiften använder jag mig av Time-series-forecasting Tutorial och samt andras notebooks.

In [ ]:
import numpy as np 
import pandas as pd 
import os
import gc
import warnings
import statsmodels.api as sm


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import matplotlib.pyplot as plt        
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.offline as offline
import plotly.graph_objs as go

pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format
warnings.filterwarnings('ignore')

# Importerar Data

In [ ]:
train = pd.read_csv("../input/store-sales-time-series-forecasting/train.csv")
test = pd.read_csv("../input/store-sales-time-series-forecasting/test.csv")
stores = pd.read_csv("../input/store-sales-time-series-forecasting/stores.csv")
transactions = pd.read_csv("../input/store-sales-time-series-forecasting/transactions.csv").sort_values(["store_nbr", "date"])

train["date"] = pd.to_datetime(train.date)
test["date"] = pd.to_datetime(test.date)
transactions["date"] = pd.to_datetime(transactions.date)

train.onpromotion = train.onpromotion.astype("float16")
train.sales = train.sales.astype("float32")
stores.cluster = stores.cluster.astype("int8")

# Transaktioner

I detta avsnitt studeras transaktioner. Hur mycket transaktioner och vid vilken tidpunkt de sker. 

### Punkter som analyseras är följande:
* Vilken veckodag som det sker flest transaktioner på.
* Vilken månad som det sker flest transaktioner på.
* Samt antalet transaktioner från de olika butikerna.

In [ ]:
transactions.head(10)

In [ ]:
temp = pd.merge(train.groupby(["date", "store_nbr"]).sales.sum().reset_index(), transactions, how = "left")
px.line(transactions.sort_values(["store_nbr", "date"]), x='date', y='transactions', color='store_nbr',title = "Antalet transaktioner per dag i de olika butikerna" )

In [ ]:
a = transactions.set_index("date").resample("M").transactions.mean().reset_index()
a["år"] = a.date.dt.year
px.line(a, x='date', y='transactions', color='år',title = "Antal transaktioner per månad" )

In [ ]:
a = transactions.copy()
a["year"] = a.date.dt.year
a["dayofweek"] = a.date.dt.dayofweek+1
a = a.groupby(["year", "dayofweek"]).transactions.mean().reset_index()
px.line(a, x="dayofweek", y="transactions" , color = "year", title = "Genomsnittligt antal transaktioner per veckodag")

## Resultat Transaktioner
Transaktionsmässigt så sker det betydligt fler transaktioner under helgen. Den månad då flest personer handlar är december månad. Ifall vi kollar på transaktioner över åren så ser vi att det inte skett en ökning i antalet transaktioner. Det var fler transaktioner under 2014 än under 2017 (Detta kan vi se genom att analysera de första månaderna under 2014 och 2017 vi ser då att det är fler under 2014). 

# Försäljning 
I detta avsnitt studeras vilken produktkategori som säljer bäst och hur försäljningen ser ut över veckor månader och år.

In [ ]:
#Sätter ihop train datan med store och transactions datan.
train = train.merge(stores, on = 'store_nbr', how='left')
train = train.merge(transactions, on = ['date', 'store_nbr'], how='left')

train['date'] = pd.to_datetime(train['date'])
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['week'] = train['date'].dt.isocalendar().week
train['quarter'] = train['date'].dt.quarter
train['day_of_week'] = train['date'].dt.day_name()
train[:5]

In [ ]:
df_st_sa = train.groupby('store_nbr').agg({"sales" : "mean"}).reset_index().sort_values(by='sales', ascending=False)
df_fa_sa = train.groupby('family').agg({"sales" : "mean"}).reset_index().sort_values(by='sales', ascending=False)[:10]
df_cl_sa = train.groupby('cluster').agg({"sales" : "mean"}).reset_index() 


fig = make_subplots(rows=2, cols=2, 
                    specs=[[{"type": "bar"}, {"type": "pie"}],
                           [{"colspan": 2}, None]],
                    column_widths=[0.7, 0.3], vertical_spacing=0, horizontal_spacing=0.02)

fig.add_trace(go.Bar(x=df_fa_sa['sales'], y=df_fa_sa['family'], marker=dict(color = '#496595'),
                     name='Family', orientation='h'), 
                     row=1, col=1)

fig.update_yaxes(showgrid=False, ticksuffix=' ', categoryorder='total ascending', row=1, col=1)
fig.update_xaxes(visible=False, row=1, col=1)
fig.update_xaxes(tickmode = 'array', tickvals=df_cl_sa.cluster, ticktext=[i for i in range(1,17)], row=2, col=1)
fig.update_yaxes(visible=False, row=2, col=1)
fig.update_layout(height=500, bargap=0.2,
                  margin=dict(b=0,r=20,l=20), xaxis=dict(tickmode='linear'),
                  title_text="Mest sålda produktkategorierna",
                  template="plotly_white",
                  title_font=dict(size=29, color='#496595'),
                  showlegend=False)
fig.show()

In [ ]:
data_grouped_day = train.groupby(['day_of_week']).mean()['sales']
data_grouped_month = train.groupby(['month']).mean()['sales']
data_grouped_year = train.groupby(['year']).mean()['sales']

plt.subplots(3,1, figsize=(20,5))
plt.subplot(131)
plt.title('sales - day')
data_grouped_day.plot(kind='bar', stacked=True)
plt.subplot(132)
plt.title('sales - month')
data_grouped_month.plot(kind='bar', stacked=True)
plt.subplot(133)
plt.title('sales - year')
data_grouped_year.plot(kind='bar', stacked=True)

## Resultat Försäljning
Försäljningsmässigt så kan vi se att de populäraste produktkategorierna är grocerys och beverages (matvaror och drycker) som tillsammans utgör ungefär 50% av försäljningen. De dagar då det säljs för mest är under lördagar och söndagar. Den månad med mest försäljning är December och man kan se en klar ökning av försäljningen mellan 2013 till 2017.

# Linjär Regression:
I detta avsnitt görs en linjär regression utav försäljningen för att förutspå hur trenden kommer att gå åren framöver.

In [ ]:
avg_sales = train.groupby('date').agg({'sales': 'mean'}).reset_index()
avg_sales['Time'] = np.arange(len(avg_sales.index))


In [ ]:
plt.style.use("seaborn-whitegrid")
plt.rc(
    "figure",
    autolayout=True,
    figsize=(12, 6),
    titlesize=18,
    titleweight='bold',
)

plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)

# Use it for the Lag_1 plot later.
plot_params = dict(
    color = '0.75',
    style = ".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    legend=False,
)

In [ ]:
avg_sales['Lag_1'] = avg_sales['sales'].shift(1)
avg_sales = avg_sales.reindex(columns = ['date','sales', 'Lag_1','Time'])
avg_sales.head()

In [ ]:
from sklearn.linear_model import LinearRegression

# Träningsdata
X = avg_sales.loc[:, ['Time']] # features
y = avg_sales.loc[:, 'sales'] # target

# Träning av min Linjära regressionsmodell
model = LinearRegression()
model.fit(X, y)

#sparar de predikterade värdena
y_pred = pd.Series(model.predict(X), index = X.index)
y_pred

In [ ]:
from pathlib import Path
from warnings import simplefilter

simplefilter("ignore")

plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True, figsize=(11, 5))
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    legend=False,
)
%config InlineBackend.figure_format = 'retina'

# försäljning average
avg_sales = train.groupby('date').agg({'sales': 'mean'}).reset_index()
avg_sales = avg_sales.set_index('date').to_period("D")

In [ ]:
from statsmodels.tsa.deterministic import DeterministicProcess

dp = DeterministicProcess(
    index=avg_sales.index,
    constant=True,       
    order=1,            
    drop=True,           
)
X = dp.in_sample()

In [ ]:
y = avg_sales["sales"]

model = LinearRegression(fit_intercept=False)
model.fit(X, y)

y_pred = pd.Series(model.predict(X), index=X.index)

In [ ]:
ax = avg_sales.plot(style=".", color="0.5", title="Försäljning - Linjär Trend")
_ = y_pred.plot(ax=ax, linewidth=3, label="Trend")

In [ ]:
X = dp.out_of_sample(steps=600)
y_fore = pd.Series(model.predict(X), index=X.index)

#Prediktering från 16 augusti 2017 och 600 dagar frammåt.
y_fore.head()

In [ ]:
ax = avg_sales["2013-01":].plot(title="Linjär Regression på Försäljning", **plot_params)
ax = y_pred["2013-01":].plot(ax=ax, linewidth=3, label="Trenden")
ax = y_fore.plot(ax=ax, linewidth=3, label="Predikterad Trend", color="C3")
_ = ax.legend()

## Resultat Linjär regression:
Vi kan se att trenden säger att det kommer bli en stabil ökning av försäljningen. 